In [1]:
import tensorflow as tf
import numpy as np
import random
import pandas as pd
import sklearn
import seaborn as sns

import matplotlib.pyplot as plt

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
import pickle

import sys
sys.path.append("../../Libraries/")
import utils
import TFT_library_masking as TFT

2024-11-12 13:07:44.955754: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-12 13:07:44.993549: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-12 13:07:44.994070: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-12 13:07:45.528856: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


# Functions

In [2]:
def load_static_data(df_static, keys, categorical_keys):
    # Eliminate the non-interesting features and get only one sample per patient
    df_static = df_static[keys].drop_duplicates()
    
    # Fill the NaNs
    if "SAPSIIIScore" in df_static:
        df_static["SAPSIIIScore"] = df_static.SAPSIIIScore.fillna(df_static.SAPSIIIScore.mean())

    # Reduce the dimensionality of two categorical features
    if 'Origin' in df_static:
        booleanMapping = (df_static.Origin == "rehabilitation") | (df_static.Origin == "hemodynamics") | \
        (df_static.Origin == "cma") | (df_static.Origin == "ICU")  | (df_static.Origin == "paediatrics")  | \
        (df_static.Origin == "obstetrics")  | (df_static.Origin == "anaesthesia")  | (df_static.Origin == "other floor") | \
         (df_static.Origin == "gynaecology") | (df_static.Origin == "psychiatry") | (df_static.Origin == "dermatology")
        df_static.Origin = df_static.Origin.where(~(booleanMapping), "others")
    if 'ReasonAdmission' in df_static:
        booleanMapping = (df_static.ReasonAdmission == "endocrine other") | (df_static.ReasonAdmission == "coagulopatía") | \
        (df_static.ReasonAdmission == "obstetric pathology") | (df_static.ReasonAdmission == "infection other")  | \
        (df_static.ReasonAdmission == "other")  | (df_static.ReasonAdmission == "hydroelectrolytic alteration")  | \
        (df_static.ReasonAdmission == "respiratory other")  | (df_static.ReasonAdmission == "severe trauma") | \
         (df_static.ReasonAdmission == "hepatic insufficiency") | (df_static.ReasonAdmission == "diabetic decompensation") | \
        (df_static.ReasonAdmission == "neuromuscular") | (df_static.ReasonAdmission == "severe arrhythmia")
        df_static.ReasonAdmission = df_static.ReasonAdmission.where(~(booleanMapping), "others")
    
    # Convert the categories of categorical features into numbers
    for i in range(len(categorical_keys)):
        df_static[categorical_keys[i]] = pd.factorize(df_static[categorical_keys[i]])[0] + 1
        
    return df_static

df_MR = pd.read_csv("../../ORIGINAL_DATA/df_estaticas_v2_covid.csv", low_memory=False)

keys = [
    'Admissiondboid', 
    'Age', 'Gender','Origin', 'ReasonAdmission', 'PatientCategory',
    'SAPSIIIScore',          
    'MonthOfAdmission', 'YearOfAdmission'
       ]
categorical_keys = ["Origin", "ReasonAdmission", "PatientCategory"]


df_static = load_static_data(df_MR, keys, categorical_keys)
df_static.head()

category_counts = []
categorical_features = ['Origin', 'ReasonAdmission', 'PatientCategory']
for i in range(len(categorical_features)):
    category_counts.append(np.unique(df_static[categorical_features[i]], return_counts=True)[0].shape[0] + 1)

In [ ]:
def myCVGrid(hyperparameters, dropout, lr_scheduler, layers, split, seed):
    bestHyperparameters = {}
    bestMetricDev = np.inf

    for k in range(len(dropout)):
        for l in range(len(layers)):
            for m in range(len(lr_scheduler)):
                v_early = []
                v_metric_dev = []
                v_hist = []
                v_val_loss = []

                hyperparameters_copy = hyperparameters.copy()
                hyperparameters_copy['dropout_rate'] = dropout[k]
                hyperparameters_copy['layers'] = layers[l]
                hyperparameters_copy['lr_scheduler'] = lr_scheduler[m]
                
                for n in range(5):

                    X_train = np.load("../../ORIGINAL_DATA/splits_14_days/notbalanced/split_" + str(i) +
                                          "/X_train_tensor_" + str(n)+ ".npy")
                    X_train_static = pd.read_csv("../../ORIGINAL_DATA/splits_14_days/notbalanced/split_" + str(i) +
                                             "/X_train_static_" + str(n)+ ".csv", index_col=0)
                    y_train = pd.read_csv("../../ORIGINAL_DATA/splits_14_days/notbalanced/split_" + str(i) +
                                          "/y_train_" + str(n)+ ".csv",
                                         index_col=0)
                    X_val = np.load("../../ORIGINAL_DATA/splits_14_days/notbalanced/split_" + str(i) +
                                        "/X_val_tensor_" + str(n)+ ".npy")
                    X_val_static = pd.read_csv("../../ORIGINAL_DATA/splits_14_days/notbalanced/split_" + str(i) +
                                             "/X_val_static_" + str(n)+ ".csv", index_col=0)
                    y_val = pd.read_csv("../../ORIGINAL_DATA/splits_14_days/notbalanced/split_" + str(i) +
                                        "/y_val_" + str(n)+ ".csv",
                                       index_col=0)

                    utils.reset_keras()
                    model, hist, early = run_network(
                        X_train, X_train_static, 
                        y_train,
                        X_val, X_val_static, 
                        y_val,
                        hyperparameters_copy,  
                        seed
                    )
                                        
                    v_early.append(early)
                    v_hist.append(hist)
                    v_val_loss.append(np.min(hist.history["val_loss"]))
                    
                metric_dev = np.mean(v_val_loss)
                if metric_dev < bestMetricDev:
                    bestMetricDev = metric_dev
                    bestHyperparameters = {
                        'dropout_rate': dropout[k],
                        'layers': layers[l],
                        'lr_scheduler': lr_scheduler[m]
                    }

    return bestHyperparameters, X_train, y_train, X_train_static, X_val, y_val, X_val_static
    

In [ ]:
def build_model(n_timesteps, mask_value, n_dynamic_features, n_static_features, category_counts, 
                hidden_layer_size, dropout_rate, num_heads, lr_scheduler):
    # Inputs.
    static_input = tf.keras.layers.Input(shape=(n_static_features))
    dynamic_input = tf.keras.layers.Input(shape=(n_timesteps, n_dynamic_features,))
    masked = tf.keras.layers.Masking(mask_value=mask_value)(dynamic_input)

    # Get the embeddings
    static_emb, dynamic_emb = TFT.get_TFT_embeddings(static_input, n_static_features, category_counts,
                                                     masked, n_dynamic_features,
                                                     hidden_layer_size)

    # Perform the FS of the static features and pass through a GRN
    static_encoder, static_weights = TFT.static_combine_and_mask(static_emb, hidden_layer_size, dropout_rate)

    static_context_h = TFT.gated_residual_network(static_encoder,
                                                    hidden_layer_size,
                                                    dropout_rate=dropout_rate,
                                                    use_time_distributed=False)


    lstm_encoder = tf.keras.layers.GRU(
        hidden_layer_size,
        dropout=dropout_rate,
        return_sequences=False,
        activation='tanh',
        use_bias=True
    )(masked,
     initial_state=[static_context_h])
    
    output = tf.keras.layers.Dense(1, activation="sigmoid")(lstm_encoder)
    
    model = tf.keras.Model([static_input, dynamic_input], [output])
    customized_loss = utils.weighted_binary_crossentropy(hyperparameters)

    myOptimizer = tf.keras.optimizers.Adam(learning_rate=lr_scheduler)
    model.compile(loss=customized_loss, optimizer=myOptimizer)
    
    return model

In [ ]:
def run_network(X_train, X_train_static, y_train, X_val, X_val_static, y_val, hyperparameters, seed):
    
    model = None
    model = build_model(
        hyperparameters["n_timesteps"], hyperparameters["maskValue"],
        hyperparameters["n_dynamic_features"], hyperparameters["n_static_features"],  hyperparameters["category_counts"], 
        hyperparameters["layers"], hyperparameters["dropout_rate"], hyperparameters["num_heads"], hyperparameters["lr_scheduler"]
    )
    try:
        earlystopping = tf.keras.callbacks.EarlyStopping(
            monitor=hyperparameters["monitor"],
            min_delta=hyperparameters["mindelta"],
            patience=hyperparameters["patience"], 
            restore_best_weights=True,
            mode="min"
        )

        hist = model.fit(
            x=[X_train_static.values, X_train], y=y_train,
            validation_data=([X_val_static.values, X_val], y_val),
            callbacks=[earlystopping], batch_size=hyperparameters['batch_size'], epochs=hyperparameters['epochs'],
            verbose=hyperparameters['verbose'])

        return model, hist, earlystopping
    except KeyboardInterrupt:
        print ('Training duration (s) : ', time.time() - global_start_time)
        return model, y_test, 0, 0

# Hyperparameters

In [ ]:
seeds = [20, 30, 45, 70]

n_categorical_features = 3
n_numerical_features = 5
n_static_features = n_categorical_features + n_numerical_features
n_dynamic_features = 56
n_timesteps = 14

# Hyperparamas of network
balance = True
epochs = 10000
batch_size = 128
num_heads = 1

layers = [3, 5, 8, 10, 15, 20, 25, 30, 35, 40, 50]
lr_scheduler = [0.0001, 0.001, 0.01, 0.1]
dropout_rate = [0.0, 0.1, 0.2, 0.3]

w2 = 0.18
w1 = 0.82

tensor = True
debug = True

hyperparameters = {
    "n_categorical_features": n_categorical_features,
    "n_numerical_features": n_numerical_features,
    "n_static_features": n_static_features,
    "n_dynamic_features": n_dynamic_features,
    "n_timesteps": n_timesteps,
    "num_heads": num_heads,
    "w1":w1, "w2":w2, 

    "category_counts": category_counts,
    "epochs":epochs,
    'batch_size': batch_size,
    'maskValue':666,
    'monitor': 'val_loss', 
    "mindelta": 0,
    "patience":30,
    'balance': balance,
    'optimizer':'adam',
    'kfold':5,
    'level':3, 
    'verbose':0
}

# Code

In [ ]:
run_model = True

if run_model:
    v_early = []
    loss_dev = []
    v_models = []
    bestHyperparameters_bySplit = {}
    y_pred_by_split = {}

    for i in range(4):
        X_test_dynamic = np.load("../../ORIGINAL_DATA/splits_14_days/notbalanced/split_" + str(i) + "/X_test_tensor.npy")
        X_test_static = pd.read_csv("../../ORIGINAL_DATA/splits_14_days/notbalanced/split_" + str(i) + "/X_test_static.csv",
                                   index_col=0)
        y_test = pd.read_csv("../../ORIGINAL_DATA/splits_14_days/notbalanced/split_" + str(i) + "/y_test.csv",
                            index_col=0)
    
        bestHyperparameters, X_train_dyn, y_train, X_train_static, X_val_dyn, y_val, X_val_static = myCVGrid(hyperparameters,
                                                                                                     dropout_rate,
                                                                                                     lr_scheduler,
                                                                                                     layers,
                                                                                                     i,                                                              
                                                                                                     seeds[i],
                                                                                                     )
        bestHyperparameters_bySplit[str(i)] = bestHyperparameters
        
    
        # Save best hyperparameters for current split
        split_directory = './Results_FHSI/split_' + str(i)
        if not os.path.exists(split_directory):
            os.makedirs(split_directory)
    
        with open(os.path.join(split_directory, f"bestHyperparameters_split_{i}.pkl"), 'wb') as f:
            pickle.dump(bestHyperparameters, f)
    
    
        hyperparameters = {
            "n_categorical_features": hyperparameters["n_categorical_features"],
            "n_numerical_features": hyperparameters["n_numerical_features"],
            "n_static_features": hyperparameters["n_static_features"],
            "n_dynamic_features": hyperparameters["n_dynamic_features"],
            "w1":hyperparameters["w1"], "w2":hyperparameters["w2"],                                    
    
            "n_timesteps": hyperparameters["n_timesteps"],
            "category_counts": hyperparameters["category_counts"],
            'epochs':  hyperparameters["epochs"],
            "num_heads": num_heads,
            'batch_size': hyperparameters["batch_size"],
            'maskValue': hyperparameters["maskValue"],
            'earlyStopping': True,
            'kfold': hyperparameters["kfold"],
            'monitor': hyperparameters["monitor"],
            "mindelta": hyperparameters["mindelta"],
            "patience": hyperparameters["patience"],
            'balance': hyperparameters["balance"],
            "dropout_rate": bestHyperparameters["dropout_rate"],
            "layers": bestHyperparameters["layers"],
            "lr_scheduler": bestHyperparameters["lr_scheduler"],
            "level": 3, 'verbose': 0
        }
    
        #Try on test
        utils.reset_keras()
    
        model, hist, early = run_network(
            X_train_dyn, X_train_static, y_train.individualMRGerm.values,
            X_val_dyn, X_val_static, y_val.individualMRGerm.values,
            hyperparameters, 
            seeds[i]
        )    
    
        v_models.append(model)
        loss_dev.append(hist.history['val_loss'])
    
        y_pred = model.predict(x=[X_test_static.values, X_test_dynamic])
        y_pred_by_split[str(i)] = y_pred
        
        with open(os.path.join(split_directory, f"y_pred_split_{i}.pkl"), 'wb') as f:
            pickle.dump(y_pred, f)
    
        # Save model for current split
        model_filename = os.path.join(split_directory, f"model_split_{i}.h5")
        model.save(model_filename)
    
        # Calculate metrics
        metrics_dict = utils.calculate_and_save_metrics(
        y_test.individualMRGerm.values, 
        y_pred, 
        split_directory, 
        split_index=i
        )